In [ ]:
import geopandas as gpd
import geemap
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Read the shapefile
shapefile_path = '../shp/AOI_02.shp'
gdf = gpd.read_file(shapefile_path)

# Convert the GeoDataFrame to GeoJSON
geojson = gdf.to_json()

# Convert the GeoJSON to an Earth Engine object
aoi = geemap.geojson_to_ee(geojson)

In [ ]:
# Load Sentinel-2 imagery
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Composite the images
composite = sentinel2.median()


# Define training data (manually created polygons for different classes)
# Example coordinates for training polygons, you will need to define your own
training_polygons = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Polygon([[longitude1, latitude1], [longitude2, latitude2], [longitude3, latitude3], [longitude4, latitude4]]), {'class': 0}),  # Fallow land
    ee.Feature(ee.Geometry.Polygon([[longitude5, latitude5], [longitude6, latitude6], [longitude7, latitude7], [longitude8, latitude8]]), {'class': 1})   # Horticulture crops
])

In [ ]:
# Sample the input imagery to get a FeatureCollection of training data
training = composite.sampleRegions(collection=training_polygons, properties=['class'], scale=10)

In [ ]:
# Train a classifier
classifier = ee.Classifier.smileRandomForest(10).train(features=training, classProperty='class', inputProperties=composite.bandNames())

# Classify the composite
classified = composite.classify(classifier)

In [ ]:
# Visualize the results using geemap
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'max': 3000}, 'Sentinel-2')
Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['blue', 'green']}, 'Classified')
Map